# Notebook for model training

### Training Flow

Load Training Data:
- Load set of input data and label files
- Prefilter positive set only to rule out heavily-masked patches
- Create train/test set
- Define augmentation parameters

Train Model:
- Define model architecture
- Compile model
- Train and evaluate model
- Save model

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from datetime import date
from datetime import datetime
import time
import os
import pickle
import sys

import pandas as pd
import io
from contextlib import redirect_stdout
import re
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from tensorflow import keras
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.models import Model

from keras.layers import Input, Dense, Flatten, Dropout, Conv2D
from keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tqdm.notebook import tqdm

parent_dir = os.path.split(os.getcwd())[0]
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)

from scripts import dl_utils
from scripts import viz_tools
from scripts import models

import gc
from tensorflow.keras import backend as K

# export TF_GPU_ALLOCATOR=cuda_malloc_async
os.environ['TF_GPU_ALLOCATOR'] = 'cuda_malloc_async'

2024-09-15 12:42:28.730918: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-15 12:42:28.810295: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-15 12:42:28.810335: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-15 12:42:28.817822: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-15 12:42:28.855107: I tensorflow/core/platform/cpu_feature_guar

## Open Data

In [3]:
resolution = 48

data_files = [
    "amazonas_2020_thresh_0.5_2_negatives_2019-01-01_2020-01-01_patch_arrays.pkl",
    "amazonas_2020_thresh_0.8_sumbsample_3_positives_2019-01-01_2020-01-01_patch_arrays.pkl",
    "MinesPos2018-2020Sentinel_points_2019-01-01_2020-01-01_patch_arrays.pkl",
    "bolivar_2020_thresh_0.8_sumbsample_5_positives_2019-01-01_2020-01-01_patch_arrays.pkl",
    "full_amazon_v9_negatives_2019-01-01_2020-01-01_patch_arrays.pkl",
    "v2.4_amazon_positives_2019-01-01_2020-01-01_patch_arrays.pkl",
    "v2.6_amazon_thresh_0.8_negatives_2019-01-01_2020-01-01_patch_arrays.pkl",
    "v2.0_bolivar_negatives_2019-01-01_2020-01-01_patch_arrays.pkl",
    "riverbank_negatives_2019-01-01_2020-01-01_patch_arrays.pkl",
    "bolivar_2020_thresh_0.8_1_negatives_2019-01-01_2020-01-01_patch_arrays.pkl",
    "v2.1.1_bolivar_negatives_2019-01-01_2020-01-01_patch_arrays.pkl",
    "v2.1_bolivar_positives_2019-01-01_2020-01-01_patch_arrays.pkl",
    "v2.4_amazonas_negatives_2019-01-01_2020-01-01_patch_arrays.pkl",
    "v2.4_amazon_negatives_2019-01-01_2020-01-01_patch_arrays.pkl",
    "v2.6_amazon_negatives_2019-01-01_2020-01-01_patch_arrays.pkl",
    "v2.6_amazon_negatives_v2_2019-01-01_2020-01-01_patch_arrays.pkl",
    "v3.1_2023_negatives_2023-01-01_2024-01-01_patch_arrays.pkl",
    "amazon_all_48px_v3.1_2023_positives_0.999_2023-01-01_2024-01-01_patch_arrays.pkl",
    "v3.2_2023_negatives_2023-01-01_2024-01-01_patch_arrays.pkl",
    "v3.3_2023_positives_2023-01-01_2024-01-01_patch_arrays.pkl",
    "v3.4_2023_negatives_2023-01-01_2024-01-01_patch_arrays.pkl",
    "v3.5_2023_negatives_2023-01-01_2024-01-01_patch_arrays.pkl",
    "v3.6_2023_positives_2023-01-01_2024-01-01_patch_arrays.pkl"
    
    ]

label_files = [
    "amazonas_2020_thresh_0.5_2_negatives_2019-01-01_2020-01-01_patch_array_labels.pkl",
    "amazonas_2020_thresh_0.8_sumbsample_3_positives_2019-01-01_2020-01-01_patch_array_labels.pkl",
    "MinesPos2018-2020Sentinel_points_2019-01-01_2020-01-01_patch_array_labels.pkl",
    "bolivar_2020_thresh_0.8_sumbsample_5_positives_2019-01-01_2020-01-01_patch_array_labels.pkl",
    "full_amazon_v9_negatives_2019-01-01_2020-01-01_patch_array_labels.pkl",
    "v2.4_amazon_positives_2019-01-01_2020-01-01_patch_array_labels.pkl",
    "v2.6_amazon_thresh_0.8_negatives_2019-01-01_2020-01-01_patch_array_labels.pkl",
    "v2.0_bolivar_negatives_2019-01-01_2020-01-01_patch_array_labels.pkl",
    "riverbank_negatives_2019-01-01_2020-01-01_patch_array_labels.pkl",
    "bolivar_2020_thresh_0.8_1_negatives_2019-01-01_2020-01-01_patch_array_labels.pkl",
    "v2.1.1_bolivar_negatives_2019-01-01_2020-01-01_patch_array_labels.pkl",
    "v2.1_bolivar_positives_2019-01-01_2020-01-01_patch_array_labels.pkl",
    "v2.4_amazonas_negatives_2019-01-01_2020-01-01_patch_array_labels.pkl",
    "v2.4_amazon_negatives_2019-01-01_2020-01-01_patch_array_labels.pkl",
    "v2.6_amazon_negatives_2019-01-01_2020-01-01_patch_array_labels.pkl",
    "v2.6_amazon_negatives_v2_2019-01-01_2020-01-01_patch_array_labels.pkl",
    "v3.1_2023_negatives_2023-01-01_2024-01-01_patch_array_labels.pkl",
    "amazon_all_48px_v3.1_2023_positives_0.999_2023-01-01_2024-01-01_patch_array_labels.pkl",
    "v3.2_2023_negatives_2023-01-01_2024-01-01_patch_array_labels.pkl",
    "v3.3_2023_positives_2023-01-01_2024-01-01_patch_array_labels.pkl",
    "v3.4_2023_negatives_2023-01-01_2024-01-01_patch_array_labels.pkl",
    "v3.5_2023_negatives_2023-01-01_2024-01-01_patch_array_labels.pkl",
    "v3.6_2023_positives_2023-01-01_2024-01-01_patch_array_labels.pkl"
    
    
]

patches = []
labels = []

data_dir = os.path.join('..', 'data', 'training_data', f"{resolution}_px")

for data, label in tqdm(zip(data_files, label_files), total=len(data_files)):
    with open(os.path.join(data_dir, data), 'rb') as f:
        data = pickle.load(f)
        for elem in data:
            #patch = dl_utils.pad_patch(elem, resolution)
            patches.append(elem)
    with open(os.path.join(data_dir, label), 'rb') as f:
        label = pickle.load(f)
        labels = np.concatenate((labels, label))

patches = np.array(patches)
negative_patches = patches[labels == 0]
positive_patches = patches[labels == 1]

print(len(patches), "samples loaded")
print(sum(labels == 1), "positive samples")
print(sum(labels == 0), "negative samples")

  0%|          | 0/23 [00:00<?, ?it/s]

4549 samples loaded
1891 positive samples
2658 negative samples


In [4]:
num_samples = 64
indices = np.random.randint(0, len(patches), num_samples)
# viz_tools.plot_image_grid(patches[indices], labels=[int(label) for label in labels[indices]])

In [5]:
def filter_black(data, mask_limit=0.1, return_rejects=False):
    masked_fraction = np.array([np.sum(np.mean(patch, axis=-1) < 10) / np.size(np.mean(patch, axis=-1)) for patch in data])
    filtered_data = data[masked_fraction < mask_limit]
    print(f"{len(filtered_data) / len(data) :.1%} of data below brightness limit")
    if return_rejects:
        rejected_data = data[masked_fraction >= mask_limit]
        return filtered_data, rejected_data
    else:
        return filtered_data

In [6]:
# Filter positive pixels that are masked beyond a threshold. Don't want to give positive examples of cloud-masked patches
filtered_positives, positive_rejects = filter_black(positive_patches, mask_limit = 0.1, return_rejects=True)
if len(positive_rejects) > 0:
    num_samples = 16**2
    if len(positive_rejects) < num_samples:
        num_samples = len(positive_rejects)
    indices = np.random.randint(0, len(positive_rejects), num_samples)
    # viz_tools.plot_numpy_grid(positive_rejects[indices,:,:,3:0:-1] / 3000)
    # plt.title("Positive Masked Rejects")
    # plt.show()

num_samples = 25 ** 2
indices = np.random.randint(0, len(filtered_positives), num_samples)
# fig = viz_tools.plot_numpy_grid(filtered_positives[indices,:,:,3:0:-1] / 3000)
# plt.title('Positive Filtered Samples')
# plt.show()

99.9% of data below brightness limit


In [7]:
filtered_negatives, negative_rejects = filter_black(negative_patches, mask_limit = 0.6, return_rejects=True)
if len(negative_rejects) > 0:
    num_samples = 16**2
    if len(positive_rejects) < num_samples:
        num_samples = len(positive_rejects)
    indices = np.random.randint(0, len(negative_rejects), num_samples)
    # viz_tools.plot_numpy_grid(negative_rejects[indices,:,:,3:0:-1] / 3000)
    # plt.title("Negative Mask Rejects")
    # plt.show()

num_samples = 25 ** 2
indices = np.random.randint(0, len(filtered_negatives), num_samples)
# fig = viz_tools.plot_numpy_grid(filtered_negatives[indices,:,:,3:0:-1] / 3000)
# plt.title('Negative Filtered Samples')
# plt.show()

100.0% of data below brightness limit


## Prepare Data for Training

In [8]:
#for RGBIR, x = normalize(np.copy(images[:,:,:,[1,2,3,8]]))
x = np.concatenate((filtered_negatives, filtered_positives))
y = np.concatenate((np.zeros(len(filtered_negatives)), np.ones(len(filtered_positives))))
x, y = shuffle(x, y, random_state=33)

In [9]:
augmentation_parameters = {
    'featurewise_center': False,
    'rotation_range': 360,
    'width_shift_range': [0.9, 1.1],
    'height_shift_range': [0.9, 1.1],
    'shear_range': 10,
    'zoom_range': [0.9, 1.1],
    'vertical_flip': True,
    'horizontal_flip': True,
    # Fill options: "constant", "nearest", "reflect" or "wrap"
    'fill_mode': 'reflect'
}

datagen = ImageDataGenerator(**augmentation_parameters)


plt.figure(figsize=(12,12), facecolor=(1,1,1), dpi=150)
aug_img, aug_labels = datagen.flow(x, y, batch_size=64).next()
# viz_tools.plot_image_grid(aug_img, labels=[int(l) for l in aug_labels], norm=True);


/home/raulpg/anaconda3/envs/tfm_tf/lib/python3.11/site-packages/keras/src/preprocessing/image.py:766: UserWarning: NumpyArrayIterator is set to use the data format convention "channels_last" (channels on axis 3), i.e. expected either 1, 3, or 4 channels on axis 3. However, it was passed an array with shape (4548, 48, 48, 12) (12 channels).
  warnings.warn(


<Figure size 1800x1800 with 0 Axes>

In [10]:
x_norm = np.clip(np.array(x.astype("float32") / 10000), 0, 1)

Partimos el 10% del dataset total para test

In [11]:
dataset_size = len(y)
test_partition = int(dataset_size*0.1)

In [12]:
num_positive_samples_train = sum(y[test_partition:]==1)
num_negative_samples_train = sum(y[test_partition:]==0)

print(f"Número de clases positivas en el dataset de training es de {num_positive_samples_train}, número de clases negativas {num_negative_samples_train}")

Número de clases positivas en el dataset de training es de 1705, número de clases negativas 2389


Eliminamos estos samples del dataset de validacion

In [13]:
test_partition_x = x_norm[:test_partition]
test_partition_y = y[:test_partition]

x_norm = x_norm[test_partition:]
y = y[test_partition:]

##  Hiperparámetros ejecuciones originales

# Preparación de hiperparámetros

## Balanceo de clases para el entrenamiento

In [14]:
from sklearn.utils import class_weight
import numpy as np

# Suponiendo que 'y_train' es un array de las etiquetas de entrenamiento
class_weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y),
    y=y
)
class_weight_dict = dict(enumerate(class_weights))

print(f"\nEl número de valores para la clase no mina es de {class_weight_dict[0]} y para la clase mina {class_weight_dict[1]}")
print(f"\nPonemos peso a cada clase para tenerlo en cuenta en el entrenamiento {class_weight_dict}")


El número de valores para la clase no mina es de 0.8568438677270824 y para la clase mina 1.2005865102639297

Ponemos peso a cada clase para tenerlo en cuenta en el entrenamiento {0: 0.8568438677270824, 1: 1.2005865102639297}


## Clases no balanceadas

In [15]:
# add a weighted loss
class_weight = {0: 1, 1: 1}

## Diccionario con todos los hiperparámetros editables

### Hiperparámetros para pruebas: early stopping = 7 y LR descendiente = 4 con test size del 20%

In [16]:
#########################################################################################################
########################################## CLASES BALANCEADAS ###########################################
#########################################################################################################
                                                                                                        #
# reduce_lr en callbacks                                                                                #                            
hiper_params_test1_20 = {"class_weight" : class_weight_dict, #
                      "batch_size" : 32,                                                                #
                      "epochs" : 160,                                                                   #
                      "define_callbacks" : ["reduce_lr", "early_stopping", "model_checkpoint"],         #
                      "patience_early_stopping" : 7,                                                   #
                      "patience_reduce_lr" : 4,                                                         #
                      "test_size" : 0.2                                                                 #
                     }                                                                                  #
                                                                                                        #
# SIN reduce_lr en callbacks                                                                            #
hiper_params_test2_20 = {"class_weight" : class_weight_dict, #
                      "batch_size" : 32,                                                                #
                      "epochs" : 160,                                                                   #
                      "define_callbacks" : ["early_stopping", "model_checkpoint"],                      #
                      "patience_early_stopping" : 7,                                                   #
                      "test_size" : 0.2                                                                 #
                     }                                                                                  #
#########################################################################################################

#########################################################################################################
####################################### CLASES NO BALANCEADAS ###########################################
#########################################################################################################
                                                                                                        #
# Reduce_lr en callbacks                                                                                #
hiper_params_test3_20 = {"class_weight" : {0: 1, 1: 1},                                                 #
                      "batch_size" : 32,                                                                #
                      "epochs" : 160,                                                                   #
                      "define_callbacks" : ["reduce_lr", "early_stopping", "model_checkpoint"],         #
                      "patience_early_stopping" : 7,                                                   #
                      "patience_reduce_lr" : 4,                                                         #
                      "test_size" : 0.2                                                                 #
                     }                                                                                  #
#SIN Reduce_lr en callbacks                                                                             #
hiper_params_test4_20 = {"class_weight" : {0: 1, 1: 1},                                                 #
                      "batch_size" : 32,                                                                #
                      "epochs" : 160,                                                                   #
                      "define_callbacks" : ["early_stopping", "model_checkpoint"],                      #
                      "patience_early_stopping" : 7,                                                   #
                      "test_size" : 0.2                                                                 #
                     }                                                                                  #
#########################################################################################################

### Hiperparámetros para pruebas: early stopping = 7 y LR descendiente = 4 con test size del 30%

In [17]:
#########################################################################################################
###################################### CLASES BALANCEADAS ###############################################
#########################################################################################################
                                                                                                        #
# Reduce_lr en callbacks                                                                                #
hiper_params_test1_30 = {"class_weight" : class_weight_dict, #
                      "batch_size" : 16,                                                                #
                      "epochs" : 160,                                                                   #
                      "define_callbacks" : ["reduce_lr", "early_stopping", "model_checkpoint"],         #
                      "patience_early_stopping" : 7,                                                   #
                      "patience_reduce_lr" : 4,                                                         #
                      "test_size" : 0.3                                                                 #
                     }                                                                                  #         
                                                                                                        #
# SIN reduce_lr en callbacks                                                                            #   
hiper_params_test2_30 = {"class_weight" : class_weight_dict, #
                      "batch_size" : 16,                                                                #
                      "epochs" : 160,                                                                   #
                      "define_callbacks" : ["early_stopping", "model_checkpoint"],                      #
                      "patience_early_stopping" : 7,                                                   #
                      "test_size" : 0.3                                                                 #
                     }                                                                                  #
#########################################################################################################

######################################################################################################
###################################### CLASES NO BALANCEADAS #########################################
######################################################################################################
                                                                                                     #
# Reduce_lr en callbacks                                                                             #  
hiper_params_test3_30 = {"class_weight" : {0: 1, 1: 1},                                              #
                      "batch_size" : 16,                                                             #
                      "epochs" : 160,                                                                #
                      "define_callbacks" : ["reduce_lr", "early_stopping", "model_checkpoint"],      #
                      "patience_early_stopping" : 7,                                                #
                      "patience_reduce_lr" : 4,                                                      #
                      "test_size" : 0.3                                                              #
                     }                                                                               #
                                                                                                     #
# SIN reduce_lr en callbacks                                                                         #      
hiper_params_test4_30 = {"class_weight" : {0: 1, 1: 1},                                              #
                      "batch_size" : 16,                                                             #
                      "epochs" : 160,                                                                #
                      "define_callbacks" : ["early_stopping", "model_checkpoint"],                   #
                      "patience_early_stopping" : 7,                                                #
                      "test_size" : 0.3                                                              #
                     }                                                                               #
######################################################################################################

### Hiperparámetros para pruebas: aumento patience en early stopping = 14 y LR descendiente = 8, split de 20%

In [18]:
#########################################################################################################
########################################## CLASES BALANCEADAS ###########################################
#########################################################################################################
                                                                                                        #
# reduce_lr en callbacks                                                                                #                            
hiper_params_test5_20 = {"class_weight" : class_weight_dict, #
                      "batch_size" : 32,                                                                #
                      "epochs" : 160,                                                                   #
                      "define_callbacks" : ["reduce_lr", "early_stopping", "model_checkpoint"],         #
                      "patience_early_stopping" : 14,                                                   #
                      "patience_reduce_lr" : 8,                                                        #
                      "test_size" : 0.2                                                                 #
                     }                                                                                  #
                                                                                                        #
# SIN reduce_lr en callbacks                                                                            #
hiper_params_test6_20 = {"class_weight" : class_weight_dict, #
                      "batch_size" : 32,                                                                #
                      "epochs" : 160,                                                                   #
                      "define_callbacks" : ["early_stopping", "model_checkpoint"],                      #
                      "patience_early_stopping" : 14,                                                   #
                      "test_size" : 0.2                                                                 #
                     }                                                                                  #
#########################################################################################################

#########################################################################################################
####################################### CLASES NO BALANCEADAS ###########################################
#########################################################################################################
                                                                                                        #
# Reduce_lr en callbacks                                                                                #
hiper_params_test7_20 = {"class_weight" : {0: 1, 1: 1},                                                 #
                      "batch_size" : 32,                                                                #
                      "epochs" : 160,                                                                   #
                      "define_callbacks" : ["reduce_lr", "early_stopping", "model_checkpoint"],         #
                      "patience_early_stopping" : 14,                                                   #
                      "patience_reduce_lr" : 8,                                                        #
                      "test_size" : 0.2                                                                 #
                     }                                                                                  #
#SIN Reduce_lr en callbacks                                                                             #
hiper_params_test8_20 = {"class_weight" : {0: 1, 1: 1},                                                 #
                      "batch_size" : 32,                                                                #
                      "epochs" : 160,                                                                   #
                      "define_callbacks" : ["early_stopping", "model_checkpoint"],                      #
                      "patience_early_stopping" : 14,                                                   #
                      "test_size" : 0.2                                                                 #
                     }                                                                                  #
#########################################################################################################

### Hiperparámetros para pruebas: aumento patience en early stopping = 14 y LR descendiente = 8, split de 30%

In [19]:
#########################################################################################################
###################################### CLASES BALANCEADAS ###############################################
#########################################################################################################
                                                                                                        #
# Reduce_lr en callbacks                                                                                #
hiper_params_test5_30 = {"class_weight" : class_weight_dict, #
                      "batch_size" : 16,                                                                #
                      "epochs" : 160,                                                                   #
                      "define_callbacks" : ["reduce_lr", "early_stopping", "model_checkpoint"],         #
                      "patience_early_stopping" : 14,                                                   #
                      "patience_reduce_lr" : 8,                                                        #
                      "test_size" : 0.3                                                                 #
                     }                                                                                  #         
                                                                                                        #
# SIN reduce_lr en callbacks                                                                            #   
hiper_params_test6_30 = {"class_weight" : class_weight_dict, #
                      "batch_size" : 16,                                                                #
                      "epochs" : 160,                                                                   #
                      "define_callbacks" : ["early_stopping", "model_checkpoint"],                      #
                      "patience_early_stopping" : 14,                                                   #
                      "test_size" : 0.3                                                                 #
                     }                                                                                  #
#########################################################################################################

######################################################################################################
###################################### CLASES NO BALANCEADAS #########################################
######################################################################################################
                                                                                                     #
# Reduce_lr en callbacks                                                                             #  
hiper_params_test7_30 = {"class_weight" : {0: 1, 1: 1},                                              #
                      "batch_size" : 16,                                                             #
                      "epochs" : 160,                                                                #
                      "define_callbacks" : ["reduce_lr", "early_stopping", "model_checkpoint"],      #
                      "patience_early_stopping" : 14,                                                #
                      "patience_reduce_lr" : 8,                                                     #
                      "test_size" : 0.3                                                              #
                     }                                                                               #
                                                                                                     #
# SIN reduce_lr en callbacks                                                                         #      
hiper_params_test8_30 = {"class_weight" : {0: 1, 1: 1},                                              #
                      "batch_size" : 16,                                                             #
                      "epochs" : 160,                                                                #
                      "define_callbacks" : ["early_stopping", "model_checkpoint"],                   #
                      "patience_early_stopping" : 14,                                                #
                      "test_size" : 0.3                                                              #
                     }                                                                               #
######################################################################################################

### Hiperparámetros para pruebas: aumento patience en early stopping = 20 y LR descendiente = 10, split de 20%

In [20]:
#########################################################################################################
########################################## CLASES BALANCEADAS ###########################################
#########################################################################################################
                                                                                                        #
# reduce_lr en callbacks                                                                                #                            
hiper_params_test9_20 = {"class_weight" : class_weight_dict, #
                      "batch_size" : 32,                                                                #
                      "epochs" : 160,                                                                   #
                      "define_callbacks" : ["reduce_lr", "early_stopping", "model_checkpoint"],         #
                      "patience_early_stopping" : 20,                                                   #
                      "patience_reduce_lr" : 10,                                                        #
                      "test_size" : 0.2                                                                 #
                     }                                                                                  #
                                                                                                        #
# SIN reduce_lr en callbacks                                                                            #
hiper_params_test10_20 = {"class_weight" : class_weight_dict, #
                      "batch_size" : 32,                                                                #
                      "epochs" : 160,                                                                   #
                      "define_callbacks" : ["early_stopping", "model_checkpoint"],                      #
                      "patience_early_stopping" : 20,                                                   #
                      "test_size" : 0.2                                                                 #
                     }                                                                                  #
#########################################################################################################

#########################################################################################################
####################################### CLASES NO BALANCEADAS ###########################################
#########################################################################################################
                                                                                                        #
# Reduce_lr en callbacks                                                                                #
hiper_params_test11_20 = {"class_weight" : {0: 1, 1: 1},                                                 #
                      "batch_size" : 32,                                                                #
                      "epochs" : 160,                                                                   #
                      "define_callbacks" : ["reduce_lr", "early_stopping", "model_checkpoint"],         #
                      "patience_early_stopping" : 20,                                                   #
                      "patience_reduce_lr" : 10,                                                        #
                      "test_size" : 0.2                                                                 #
                     }                                                                                  #
#SIN Reduce_lr en callbacks                                                                             #
hiper_params_test12_20 = {"class_weight" : {0: 1, 1: 1},                                                 #
                      "batch_size" : 32,                                                                #
                      "epochs" : 160,                                                                   #
                      "define_callbacks" : ["early_stopping", "model_checkpoint"],                      #
                      "patience_early_stopping" : 14,                                                   #
                      "test_size" : 0.2                                                                 #
                     }                                                                                  #
#########################################################################################################

### Hiperparámetros para pruebas: aumento patience en early stopping = 20 y LR descendiente = 10, split de 30%

In [21]:
#########################################################################################################
###################################### CLASES BALANCEADAS ###############################################
#########################################################################################################
                                                                                                        #
# Reduce_lr en callbacks                                                                                #
hiper_params_test9_30 = {"class_weight" : class_weight_dict, #
                      "batch_size" : 16,                                                                #
                      "epochs" : 160,                                                                   #
                      "define_callbacks" : ["reduce_lr", "early_stopping", "model_checkpoint"],         #
                      "patience_early_stopping" : 20,                                                   #
                      "patience_reduce_lr" : 10,                                                        #
                      "test_size" : 0.3                                                                 #
                     }                                                                                  #         
                                                                                                        #
# SIN reduce_lr en callbacks                                                                            #   
hiper_params_test10_30 = {"class_weight" : class_weight_dict, #
                      "batch_size" : 16,                                                                #
                      "epochs" : 160,                                                                   #
                      "define_callbacks" : ["early_stopping", "model_checkpoint"],                      #
                      "patience_early_stopping" : 14,                                                   #
                      "test_size" : 0.3                                                                 #
                     }                                                                                  #
#########################################################################################################

######################################################################################################
###################################### CLASES NO BALANCEADAS #########################################
######################################################################################################
                                                                                                     #
# Reduce_lr en callbacks                                                                             #  
hiper_params_test11_30 = {"class_weight" : {0: 1, 1: 1},                                              #
                      "batch_size" : 16,                                                             #
                      "epochs" : 160,                                                                #
                      "define_callbacks" : ["reduce_lr", "early_stopping", "model_checkpoint"],      #
                      "patience_early_stopping" : 20,                                                #
                      "patience_reduce_lr" : 10,                                                     #
                      "test_size" : 0.3                                                              #
                     }                                                                               #
                                                                                                     #
# SIN reduce_lr en callbacks                                                                         #      
hiper_params_test12_30 = {"class_weight" : {0: 1, 1: 1},                                              #
                      "batch_size" : 16,                                                             #
                      "epochs" : 160,                                                                #
                      "define_callbacks" : ["early_stopping", "model_checkpoint"],                   #
                      "patience_early_stopping" : 20,                                                #
                      "test_size" : 0.3                                                              #
                     }                                                                               #
######################################################################################################

### Generamos una lista con todos los hiperparámetros definidos

In [22]:
hiperparametros_20 = [hiper_params_test1_20, hiper_params_test2_20, hiper_params_test3_20, hiper_params_test4_20]
hiperparametros_30 = [hiper_params_test1_30, hiper_params_test2_30, hiper_params_test3_30, hiper_params_test4_30]

In [23]:
hiperparametros_20_modified = [hiper_params_test5_20, hiper_params_test6_20, hiper_params_test7_20, hiper_params_test8_20]
hiperparametros_30_modified = [hiper_params_test5_30, hiper_params_test6_30, hiper_params_test7_30, hiper_params_test8_30]

In [24]:
hiperparametros_20_max_patience = [hiper_params_test9_20, hiper_params_test10_20, hiper_params_test11_20, hiper_params_test12_20]
hiperparametros_30_max_patience = [hiper_params_test9_30, hiper_params_test10_30, hiper_params_test11_30, hiper_params_test12_30]

# Entrenamiento de las diferentes arquitecturas

In [25]:
print(f"Total de modelos construidos = 6 pruebas * 4 hiperparámetros/prueba * 2 semillas distintas * 4 arquitecturas distintas = {6*4*2*4} modelos construidos. 48 modelos por cada arquitectura.")

Total de modelos construidos = 6 pruebas * 4 hiperparámetros/prueba * 2 semillas distintas * 4 arquitecturas distintas = 192 modelos construidos. 48 modelos por cada arquitectura.


In [26]:
tests = [
    # ["split_20",hiperparametros_20],
    # ["split_30",hiperparametros_30],
    # ["split_20_increase_patience",hiperparametros_20_modified],
    # ["split_30_increase_patience",hiperparametros_30_modified],
    # ["split_20_max_patience",hiperparametros_20_max_patience],
    ["split_30_max_patience",hiperparametros_30_max_patience]
]
input_shape = (48,48,12)

In [27]:
%%time

for test in tests:

    # Descripcion de esta prueba
    desc_test = test[0]
    hiperparametros = test[1]
    
    # Inicializamos para dar nombre a todos los outputs que se escriban
    hora_exec = datetime.now().strftime('%Y-%m-%d_%H-%M')
    
    # Creamos el nombre del directorio para guardar los modelos finales
    path_prueba_actual = f"../notebooks/best_models/{desc_test}_{hora_exec}/"
    os.makedirs(path_prueba_actual, exist_ok=True)
    
    # Inicializamos las variables dónde almacenaremos los resultados de cada uno de los modelos generados
    df_alexnet = pd.DataFrame()
    df_scratch_modified = pd.DataFrame()
    df_vgg = pd.DataFrame()
    df_mobile_net = pd.DataFrame()
    
    #Eliminamos los csvs generados en ejecuciones anteriores
    dl_utils.limpia_directorios("aux_path")
    
    # Recorremos la lista de configuraciones distinta generada anteriormente
    for id_hiperparams,hiperparams in enumerate(hiperparametros):
        for semilla in range(0,2):  
    
            # Limpiamos la carpeta donde se guardan los modelos
            dl_utils.limpia_directorios("weigths")
    
            # Definimos hiperparámetros asociados a la ejecución actual
            batch_size = hiperparams["batch_size"]
            epochs = hiperparams["epochs"]
         
            # Dividimos el conjunto de imágenes en train y test
            seed = np.random.randint(0, 2000,1)[0]
            hiperparams["semilla"] = seed
            keras.utils.set_random_seed(int(seed))
            x_train, x_test, y_train, y_test = train_test_split(x_norm, y, test_size=hiperparams["test_size"], random_state=123)
    
            # Generamos los datagen de imágenes, empleamos las mismas imágenes en todos los modelos
            datagen.fit(x_train)
            train_generator = datagen.flow(x_train, y_train, batch_size=batch_size, shuffle=False)
            test_generator = datagen.flow(x_test, y_test, batch_size=batch_size, shuffle=False)
            
            # Definimos y compilamos los modelos
            alexnet = models.alexnet_model(input_shape)
            scratch_modified_model = models.scratch_modified_model(input_shape)
            vgg16 = models.vgg_16_model(input_shape)
            mobile_net = models.mobilenet_model(input_shape)

            # Modelo por defecto empleando los mismos parámetros que el original.
            start_time = time.time()
            alexnet_results = alexnet.fit(
            # datagen.flow(x_train, y_train),
                train_generator,
                batch_size=batch_size,
                epochs=epochs,
                validation_data=test_generator,
                # validation_data=(x_test, y_test),
                verbose=1,
                shuffle=True,
                callbacks=dl_utils.define_callbacks("model_alexnet_",hiperparams),
                class_weight=hiperparams["class_weight"],
            )
            training_time_alexnet = time.time() - start_time
    
            # Modelo from scratch modificado, ampliación del modelo original
            start_time = time.time()
            scratch_modified_model_results = scratch_modified_model.fit(
                # datagen.flow(x_train, y_train),
                train_generator,
                batch_size=batch_size,
                epochs=epochs,
                validation_data=test_generator,
                # validation_data=(x_test, y_test),
                verbose=1,
                shuffle=True,
                callbacks=dl_utils.define_callbacks("model_scratch_modified_",hiperparams),
                class_weight=hiperparams["class_weight"],
            )
            training_time_scratch_modified_model = time.time() - start_time
    
            # Fine-tunning con los pesos del modelo VGG16
            start_time = time.time()
            vgg16_results = vgg16.fit(
                # datagen.flow(x_train, y_train),
                train_generator,
                batch_size=batch_size,
                epochs=epochs,
                validation_data=test_generator,
                # validation_data=(x_test, y_test),
                verbose=1,
                shuffle=True,
                callbacks=dl_utils.define_callbacks("model_vgg16_",hiperparams),
                class_weight=hiperparams["class_weight"],
            )
            training_time_vgg16 = time.time() - start_time
    
            # Fine-tunning con los pesos del modelo MobileNet
            start_time = time.time()
            mobile_net_results = mobile_net.fit(
                # datagen.flow(x_train, y_train),
                train_generator,
                batch_size=batch_size,
                epochs=epochs,
                validation_data=test_generator,
                # validation_data=(x_test, y_test),
                verbose=1,
                shuffle=True,
                callbacks=dl_utils.define_callbacks("model_mobilenet_",hiperparams),
                class_weight=hiperparams["class_weight"],
            )
            training_time_mobile_net = time.time() - start_time
        
            # Evaluamos el modelo y lo comparamos con el mejor almacenado.  
            hora_actual = datetime.now().strftime('%Y-%m-%d_%H-%M')
            df_alexnet,           save_df_alexnet           = dl_utils.get_df_best_model(alexnet,test_partition_x,test_partition_y,f"alexnet_{desc_test}_",df_alexnet,hora_actual)
            df_scratch_modified, save_df_scratch_modified = dl_utils.get_df_best_model(scratch_modified_model,test_partition_x,test_partition_y,f"scratch_modified_model_{desc_test}_",df_scratch_modified,hora_actual)
            df_vgg,              save_df_vgg              = dl_utils.get_df_best_model(vgg16,test_partition_x,test_partition_y,f"vgg16_{desc_test}_",df_vgg,hora_actual)
            df_mobile_net,       save_df_mobile_net       = dl_utils.get_df_best_model(mobile_net,test_partition_x,test_partition_y,f"mobile_net_{desc_test}_",df_mobile_net,hora_actual)
        
            # Si el modelo actual es mejor que el almacenado, guardamos su información.      
            if save_df_alexnet:
                best_alexnet_model = alexnet
                best_alexnet_results = alexnet_results.history
                best_alexnet_hiperparams = [hiperparams,
                                            id_hiperparams+1,
                                            f'resultados_evaluacion_alexnet_{desc_test}_{hora_actual}.csv',
                                            training_time_alexnet,
                                            len(alexnet_results.epoch)]

            if save_df_scratch_modified:
                best_scratch_modified_model = scratch_modified_model
                best_scratch_modified_model_results = scratch_modified_model_results.history
                best_scratch_modified_hiperparams = [hiperparams,
                                                     id_hiperparams+1,
                                                     f'resultados_evaluacion_scratch_modified_model_{desc_test}_{hora_actual}.csv',
                                                     training_time_scratch_modified_model,
                                                     len(scratch_modified_model_results.epoch)]
        
            if save_df_vgg:
                best_vgg_model = vgg16
                best_vgg_model_results = vgg16_results.history
                best_vgg_hiperparams =[hiperparams,
                                       id_hiperparams+1,
                                       f'resultados_evaluacion_vgg16_{desc_test}_{hora_actual}.csv',
                                       training_time_vgg16,
                                       len(vgg16_results.epoch)]
        
            if save_df_mobile_net:
                best_mobile_net_model = mobile_net
                best_mobile_net_model_results = mobile_net_results.history
                best_mobile_net_hiperparams = [hiperparams,
                                               id_hiperparams+1,
                                               f'resultados_evaluacion_mobile_net_{desc_test}_{hora_actual}.csv',
                                               training_time_mobile_net,
                                               len(mobile_net_results.epoch)]

            # Limpiamos variables auxiliares
            del alexnet
            del scratch_modified_model
            del vgg16
            del mobile_net
            del alexnet_results
            del scratch_modified_model_results
            del vgg16_results
            del mobile_net_results
            del train_generator
            del test_generator
            K.clear_session()
            gc.collect()        
    
    # Guardamos el mejor modelo
    dl_utils.save_models(best_alexnet_model,
                         best_alexnet_results,
                         best_alexnet_hiperparams,
                         hora_exec,
                         f'alexnet_{desc_test}',
                         path_prueba_actual)
    dl_utils.save_models(best_scratch_modified_model,
                         best_scratch_modified_model_results,
                         best_scratch_modified_hiperparams,
                         hora_exec,
                         f'scratch_modified_model_{desc_test}',
                         path_prueba_actual)
    dl_utils.save_models(best_vgg_model,
                         best_vgg_model_results,
                         best_vgg_hiperparams,
                         hora_exec,
                         f'vgg16_{desc_test}',
                         path_prueba_actual)
    dl_utils.save_models(best_mobile_net_model,
                         best_mobile_net_model_results,
                         best_mobile_net_hiperparams,
                         hora_exec
                         ,f'mobile_net_{desc_test}',
                         path_prueba_actual)
    
    # Movemos los archivos asociados a los resultados
    dl_utils.obtener_tablas_resultado(path_prueba_actual,[best_alexnet_hiperparams[2],
                                                         best_scratch_modified_hiperparams[2],
                                                         best_vgg_hiperparams[2],
                                                         best_mobile_net_hiperparams[2]])
    # Limpiamos variables auxiliares
    del best_alexnet_model
    del best_alexnet_results
    del best_alexnet_hiperparams
    del best_scratch_modified_model
    del best_scratch_modified_model_results
    del best_scratch_modified_hiperparams
    del best_vgg_model
    del best_vgg_model_results
    del best_vgg_hiperparams
    del best_mobile_net_model
    del best_mobile_net_model_results
    del best_mobile_net_hiperparams
    K.clear_session()
    gc.collect() 

/home/raulpg/anaconda3/envs/tfm_tf/lib/python3.11/site-packages/keras/src/preprocessing/image.py:2093: UserWarning: Expected input to be images (as Numpy array) following the data format convention "channels_last" (channels on axis 3), i.e. expected either 1, 3 or 4 channels on axis 3. However, it was passed an array with shape (2865, 48, 48, 12) (12 channels).
  warnings.warn(
/home/raulpg/anaconda3/envs/tfm_tf/lib/python3.11/site-packages/keras/src/preprocessing/image.py:766: UserWarning: NumpyArrayIterator is set to use the data format convention "channels_last" (channels on axis 3), i.e. expected either 1, 3, or 4 channels on axis 3. However, it was passed an array with shape (2865, 48, 48, 12) (12 channels).
  warnings.warn(
/home/raulpg/anaconda3/envs/tfm_tf/lib/python3.11/site-packages/keras/src/preprocessing/image.py:766: UserWarning: NumpyArrayIterator is set to use the data format convention "channels_last" (channels on axis 3), i.e. expected either 1, 3, or 4 channels on axi

Epoch 1/160


2024-09-15 12:42:42.261545: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8907
2024-09-15 12:42:42.422769: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-09-15 12:42:42.872091: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-09-15 12:42:43.514237: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f2d0c838110 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-09-15 12:42:43.514265: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 4060 Ti, Compute Capability 8.9
2024-09-15 12:42:43.518436: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1726396963.574131  658214 device_compil

179/180 [============================>.] - ETA: 0s - loss: 0.6483 - acc: 0.6135
Epoch 1: val_loss improved from inf to 0.60025, saving model to /mnt/c/Users/raulp/Documents/Master/TFM/minig-detector-tfm/notebooks/weigths/model_alexnet_weights.h5
180/180 [==============================] - 9s 31ms/step - loss: 0.6484 - acc: 0.6129 - val_loss: 0.6002 - val_acc: 0.6208 - lr: 3.0000e-04
Epoch 2/160
180/180 [==============================] - ETA: 0s - loss: 0.5639 - acc: 0.7271
Epoch 2: val_loss improved from 0.60025 to 0.53558, saving model to /mnt/c/Users/raulp/Documents/Master/TFM/minig-detector-tfm/notebooks/weigths/model_alexnet_weights.h5
180/180 [==============================] - 5s 27ms/step - loss: 0.5639 - acc: 0.7271 - val_loss: 0.5356 - val_acc: 0.7258 - lr: 3.0000e-04
Epoch 3/160
179/180 [============================>.] - ETA: 0s - loss: 0.5591 - acc: 0.7280
Epoch 3: val_loss improved from 0.53558 to 0.50188, saving model to /mnt/c/Users/raulp/Documents/Master/TFM/minig-detector

2024-09-15 12:52:16.131033: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential_1/dropout_2/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


...

Epoch 1/160
179/180 [============================>.] - ETA: 0s - loss: 0.6782 - acc: 0.5784
Epoch 1: val_loss improved from inf to 0.68149, saving model to /mnt/c/Users/raulp/Documents/Master/TFM/minig-detector-tfm/notebooks/weigths/model_alexnet_weights.h5
180/180 [==============================] - 8s 38ms/step - loss: 0.6779 - acc: 0.5787 - val_loss: 0.6815 - val_acc: 0.5354
Epoch 2/160
179/180 [============================>.] - ETA: 0s - loss: 0.6745 - acc: 0.6044
Epoch 2: val_loss improved from 0.68149 to 0.67698, saving model to /mnt/c/Users/raulp/Documents/Master/TFM/minig-detector-tfm/notebooks/weigths/model_alexnet_weights.h5
180/180 [==============================] - 7s 36ms/step - loss: 0.6746 - acc: 0.6042 - val_loss: 0.6770 - val_acc: 0.5899
Epoch 3/160
179/180 [============================>.] - ETA: 0s - loss: 0.6793 - acc: 0.5784
Epoch 3: val_loss improved from 0.67698 to 0.61900, saving model to /mnt/c/Users/raulp/Documents/Master/TFM/minig-detector-tfm/notebooks/weigths

2024-09-15 15:45:30.473533: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential_1/dropout_2/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


177/180 [============================>.] - ETA: 0s - loss: 0.6762 - acc: 0.5974
Epoch 1: val_loss improved from inf to 0.62873, saving model to /mnt/c/Users/raulp/Documents/Master/TFM/minig-detector-tfm/notebooks/weigths/model_scratch_modified_weights.h5
180/180 [==============================] - 5s 18ms/step - loss: 0.6752 - acc: 0.5993 - val_loss: 0.6287 - val_acc: 0.7185
Epoch 2/160
178/180 [============================>.] - ETA: 0s - loss: 0.5446 - acc: 0.7466
Epoch 2: val_loss improved from 0.62873 to 0.54267, saving model to /mnt/c/Users/raulp/Documents/Master/TFM/minig-detector-tfm/notebooks/weigths/model_scratch_modified_weights.h5
180/180 [==============================] - 3s 19ms/step - loss: 0.5449 - acc: 0.7455 - val_loss: 0.5427 - val_acc: 0.7087
Epoch 3/160
178/180 [============================>.] - ETA: 0s - loss: 0.5239 - acc: 0.7455
Epoch 3: val_loss improved from 0.54267 to 0.49442, saving model to /mnt/c/Users/raulp/Documents/Master/TFM/minig-detector-tfm/notebooks/w

/mnt/c/Users/raulp/Documents/Master/TFM/minig-detector-tfm/scripts/dl_utils.py:171: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  valores.append(float(df.iloc[3][-2]))
/mnt/c/Users/raulp/Documents/Master/TFM/minig-detector-tfm/scripts/dl_utils.py:175: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  valores_old.append(float(prev_df.iloc[3][-2]))
/mnt/c/Users/raulp/Documents/Master/TFM/minig-detector-tfm/scripts/dl_utils.py:171: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a va

CPU times: user 4h 27min 30s, sys: 19min 30s, total: 4h 47min
Wall time: 3h 14min 9s
